In [16]:
import cv2
import numpy as np

In [17]:
def read_images(img1_path, img2_path):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    return img1, img2

def detect_and_match_features(img1, img2):
    # 转换为灰度图像
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # 使用ORB检测关键点和描述符
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # 使用BFMatcher进行特征匹配
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # 根据距离排序匹配点
    matches = sorted(matches, key=lambda x: x.distance)
    return keypoints1, keypoints2, matches

In [18]:
def find_homography(keypoints1, keypoints2, matches):
    # 提取匹配点
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 2)

    # 计算单应性矩阵并过滤异常点
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H, mask

def warp_image(img1, img2, H):
    height, width, channels = img2.shape
    img1_warp = cv2.warpPerspective(img1, H, (width, height))
    return img1_warp

In [19]:
img1_path = 'x.jpg'
img2_path = 'y.jpg'

# 读取图片
img1, img2 = read_images(img1_path, img2_path)

# 检测和匹配特征
keypoints1, keypoints2, matches = detect_and_match_features(img1, img2)

# 可视化匹配结果
img_matches = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imwrite('matches.jpg', img_matches)

if len(matches) > 4:
    # 计算单应性矩阵
    H, mask = find_homography(keypoints1, keypoints2, matches)
        
    if H is not None:
        # 打印单应性矩阵
        print("Homography matrix:\n", H)

        # 将img1中的点投影到img2中
        img1_warp = warp_image(img1, img2, H)

        # 保存变换后的图片
        cv2.imwrite('warped_image.jpg', img1_warp)
    else:
        print("Homography calculation failed.")
else:
    print("Not enough matches are found - {}/{}".format(len(matches), 4))

print("Processing complete. Results saved as 'matches.jpg' and 'warped_image.jpg'.")

Homography matrix:
 [[ 1.25498219e-01  1.08953939e-01  3.81390627e+02]
 [-1.04474658e+00  5.02597690e-01  1.15554058e+03]
 [-3.10625211e-04 -2.99185477e-05  1.00000000e+00]]
Processing complete. Results saved as 'matches.jpg' and 'warped_image.jpg'.
